<a href="https://colab.research.google.com/github/misbakhanum/SCT_ML_3/blob/main/SCT_T3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import zipfile

zip_file_name = '/content/test1.zip'  # Replace with your actual file name
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall('/content/extracted_dataset')


In [4]:
import zipfile

zip_file_name = '/content/train.zip'  # Replace with your actual file name
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall('/content/Cat_dataset')

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Define paths to your dataset
cats_folder = '/content/Cat_dataset'
dogs_folder = '/content/extracted_dataset'

# Function to load images recursively from folder (filter only '.jpg' files)
def load_images_from_folder(folder, label):
    images = []
    labels = []
    valid_extensions = ('.jpg', '.jpeg', '.png')  # Add more extensions if needed
    for root, dirs, files in os.walk(folder):  # Recursively search all subdirectories
        for filename in files:
            if filename.lower().endswith(valid_extensions):  # Check for valid image extensions
                img_path = os.path.join(root, filename)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
                if img is not None:  # Check if the image is loaded successfully
                    img = cv2.resize(img, (64, 64))  # Resize to 64x64 pixels
                    images.append(img.flatten())  # Flatten the image into a 1D array
                    labels.append(label)
                else:
                    print(f"Failed to load image: {img_path}")
            else:
                print(f"Skipped non-image file: {filename}")
    return images, labels

# Load images
cat_images, cat_labels = load_images_from_folder(cats_folder, label=0)  # 0 for cats
dog_images, dog_labels = load_images_from_folder(dogs_folder, label=1)  # 1 for dogs

# Check if any images were loaded
if len(cat_images) == 0 or len(dog_images) == 0:
    print("No images were loaded. Please check the folder paths or file types.")
else:
    # Combine the datasets
    X = np.array(cat_images + dog_images)
    y = np.array(cat_labels + dog_labels)

    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale the data to normalize the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Create the SVM model and train it
    svm_model = SVC(kernel='linear')  # You can change kernel to 'rbf', 'poly', etc.
    svm_model.fit(X_train, y_train)

    # Make predictions
    y_pred = svm_model.predict(X_test)

    # Evaluate the model
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
